<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# Google GenAI SDK - Building a Parallelization Agent

## Install Dependencies

In [ ]:
!pip install -q google-genai arize-phoenix-otel openinference-instrumentation-google-genai

## Connect to Arize Phoenix

In [ ]:
import os
from getpass import getpass

from google import genai

from phoenix.otel import register

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

tracer_provider = register(auto_instrument=True, project_name="google-genai-parallelization-agent")
tracer = tracer_provider.get_tracer(__name__)

## Authenticate with Google Vertex AI

In [ ]:
!gcloud auth login

In [ ]:
# Create a client using the Vertex AI API, you could also use the Google GenAI API instead here
client = genai.Client(vertexai=True, project="<ADD YOUR GCP PROJECT ID>", location="us-central1")

# Parallelization

In [ ]:
import concurrent.futures
import textwrap

from opentelemetry.context import attach, detach, get_current

GEMINI_MODEL_NAME = "gemini-2.0-flash-001"


def define_research_prompts():
    """Define the research prompts for different topics."""
    return {
        "ai_research_result": (
            "Artificial Intelligence",
            """You are an AI Research Assistant.
Research the latest advancements in 'Artificial Intelligence'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
        ),
        "quantum_research_result": (
            "Quantum Computing",
            """You are an AI Research Assistant specializing in physics and computing.
Research the latest breakthroughs in 'Quantum Computing'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
        ),
        "biotech_research_result": (
            "Biotechnology",
            """You are an AI Research Assistant specializing in life sciences.
Research the latest innovations in 'Biotechnology'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
        ),
    }


@tracer.chain()
def run_research_task(topic, prompt):
    """Calls the generative model and returns the text result."""
    print(f"Starting research for: {topic}...")
    try:
        response = client.models.generate_content(model=GEMINI_MODEL_NAME, contents=prompt)
        print(f"Finished research for: {topic}.")
        return response.text.strip()
    except Exception as e:
        print(f"Error during research for {topic}: {e}")
        return f"Error retrieving information for {topic}."


# This is a helper function to wrap the run_research_task function with the OTel context.
# Without this, each research subtask would be traced as a new span, and wouldn't be nested
# under the main research task span.
def context_wrapped_task(context, func, *args, **kwargs):
    token = attach(context)
    try:
        return func(*args, **kwargs)
    finally:
        detach(token)


@tracer.chain()
def execute_parallel_research(research_prompts):
    """Execute research tasks in parallel and return the results."""
    research_results = {}
    context = get_current()  # Capture the current OTel context
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit tasks, wrapping each with the context
        future_to_key = {
            executor.submit(context_wrapped_task, context, run_research_task, topic, prompt): key
            for key, (topic, prompt) in research_prompts.items()
        }
        # Collect results as they complete
        for future in concurrent.futures.as_completed(future_to_key):
            key = future_to_key[future]
            try:
                result = future.result()
                research_results[key] = result
            except Exception as exc:
                print(f"{key} generated an exception: {exc}")
                research_results[key] = f"Error in {key} task."
    return research_results


@tracer.chain()
def create_synthesis_prompt(research_results):
    """Create a prompt for synthesizing the research results."""
    return f"""You are an AI Assistant responsible for combining research findings into a structured report.

Your primary task is to synthesize the following research summaries, clearly attributing findings to their
source areas (AI, Quantum Computing, Biotechnology). Structure your response using headings for each topic.
Ensure the report is coherent and integrates the key points smoothly.

**Crucially: Your entire response MUST be grounded *exclusively* on the information provided in the
'Input Summaries' below. Do NOT add any external knowledge, facts, or details not present in these
specific summaries.**

**Input Summaries:**

*   **AI Advancements:** {research_results.get('ai_research_result', 'N/A')}
*   **Quantum Computing:** {research_results.get('quantum_research_result', 'N/A')}
*   **Biotechnology:** {research_results.get('biotech_research_result', 'N/A')}

Produce the final synthesized report.
"""


@tracer.chain()
def synthesize_results(synthesis_instruction):
    """Generate a synthesized report from the research results."""
    print("\n--- Starting Synthesis ---")
    try:
        synthesis_response = client.models.generate_content(
            model=GEMINI_MODEL_NAME, contents=synthesis_instruction
        )
        final_report = synthesis_response.text.strip()
        print("--- Synthesis Complete ---")
        return final_report
    except Exception as e:
        print(f"Error during synthesis: {e}")
        return "Error generating the final report."


def display_report(final_report):
    """Display the final synthesized report."""
    print("\n=== Final Synthesized Report ===\n")
    # Use textwrap for potentially long reports in notebooks
    print(textwrap.fill(final_report, width=80))


# Main execution flow
@tracer.agent()
def main():
    research_prompts = define_research_prompts()
    research_results = execute_parallel_research(research_prompts)

    print("\n--- Parallel Research Results ---")
    for key, result in research_results.items():
        print(f"{key}: {result}")

    synthesis_instruction = create_synthesis_prompt(research_results)
    final_report = synthesize_results(synthesis_instruction)
    display_report(final_report)
    return final_report


main()